# Exploratory Data Analysis(EDA): Data Science Project (part IV)



#### 3) Create a python script to extract relevant data from multiple data sources, automating the process of data ingestion.

In [ ]:
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as pyoff
import seaborn as sns


%matplotlib inline


pyoff.init_notebook_mode()

##### Data Ingestion and Preprocessing

In order to leverage data to solve the business problem at hand, it is first required to be understood. The excel file is read in order to know the informations that are available in the data. 

In [ ]:
df = pd.read_excel("online_retail.xlsx")
df.head()

The various features availabe in the data are "InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "UnitPrice" and "Country" alongwith a unique CustomerID.

Let's begin by describing our data first

In [ ]:
df.describe()

In [ ]:
df.shape

Revenue (monthly) = Monthly Invoice Count * Quantity * Unit Price

In [ ]:
df["InvoiceYearMonth"] = df["InvoiceDate"].map(
    lambda date: 100 * date.year + date.month
)

df["Revenue"] = df["Quantity"] * df["UnitPrice"]
df_revenue = df.groupby(["InvoiceYearMonth"]).agg({"Revenue": sum}).reset_index()
df_revenue

#### 4) Investigate the relationship between the relevant data, the target and the business metric.

#### Exploratory Data Analysis

It is during the Exploratory Data Analysis (EDA) process that data integrity issues are identified sometimes.

After extracting data it is important to include checks for quality assurance even on the first pass through the AI workflow.  Let's combine the data into a single structure and provide a couple checks for quality assurance.

**Implementation of checks for Quality Assurance**

- Remove any repeat customers based on customer_id
- Check for missing values

In [ ]:
print("\nCleaning Summary\n{}".format("-" * 35))
print("Total records:", df.shape[0])
duplicate_rows = df.duplicated()
if True in duplicate_rows:
    df = df[~duplicate_rows]
print(f"Removed {np.where(duplicate_rows == True)[0].size} duplicate rows")

print("\nMissing Value Summary\n{}".format("-" * 35))
print("\ndf_total\n{}".format("-" * 15))
print(df.isnull().sum(axis=0))

##### Visualizing monthly revenue

In [ ]:
plot_rev(df_revenue, "InvoiceYearMonth", "Revenue", "category", "Montly Revenue")

##### Monthly growth rate

In [ ]:
# using pct_change() function to see monthly percentage change
df_revenue["MonthlyGrowth"] = df_revenue["Revenue"].pct_change()
df_revenue.head()

In [ ]:
x = df_revenue.query("InvoiceYearMonth < 201112")["InvoiceYearMonth"]
y = df_revenue.query("InvoiceYearMonth < 201112")["MonthlyGrowth"]
query_plot(x, y, "category", "Monthly Growth Rate")

##### Creating monthly active customers dataframe for UK

In [ ]:
df_uk = df.query("Country=='United Kingdom'").reset_index(drop=True)

In [ ]:
df_monthly_active = (
    df_uk.groupby("InvoiceYearMonth")["CustomerID"].nunique().reset_index()
)
df_monthly_active

In [ ]:
plot_rev(
    df_monthly_active,
    "InvoiceYearMonth",
    "CustomerID",
    "category",
    "Monthly Active Customers",
    go.Bar,
)

##### Visualizing Monthly orders

In [ ]:
df_monthly_orders = df_uk.groupby("InvoiceYearMonth")["Quantity"].sum().reset_index()
df_monthly_orders

In [ ]:
plot_rev(
    df_monthly_orders,
    "InvoiceYearMonth",
    "Quantity",
    "category",
    "Monthly Orders",
    go.Bar,
)

##### Average Revenue per Order

In [ ]:
df_monthly_revenue = df_uk.groupby("InvoiceYearMonth")["Revenue"].mean().reset_index()
df_monthly_revenue

In [ ]:
plot_rev(
    df_monthly_revenue,
    "InvoiceYearMonth",
    "Revenue",
    "category",
    "Monthly Orders",
    go.Bar,
)

In [ ]:
df_min_purchase = df_uk.groupby("CustomerID")["InvoiceDate"].min().reset_index()
df_min_purchase.columns = ["CustomerID", "MinPurchaseDate"]
df_min_purchase["MinPurchaseYearMonth"] = df_min_purchase["MinPurchaseDate"].map(
    lambda date: 100 * date.year + date.month
)
df_min_purchase.head()

In [ ]:
df_uk = pd.merge(df_uk, df_min_purchase, on="CustomerID")
df_uk.head()

##### Comparing new vs Existing

In [ ]:
df_uk["UserType"] = "New"
df_uk.loc[
    df_uk["InvoiceYearMonth"] > df_uk["MinPurchaseYearMonth"], "UserType"
] = "Existing"

In [ ]:
df_user_type_revenue = (
    df_uk.groupby(["InvoiceYearMonth", "UserType"])["Revenue"].sum().reset_index()
)
# remove december month due to incomplete data in that month
df_user_type_revenue = df_user_type_revenue.query(
    "InvoiceYearMonth != 201012 and InvoiceYearMonth != 201112"
)
df_user_type_revenue

In [ ]:
x = df_user_type_revenue.query("UserType == 'Existing'")["InvoiceYearMonth"]
y = df_user_type_revenue.query("UserType == 'Existing'")["Revenue"]
query_plot(x, y, "category", "New vs Existing")

##### New Customer Ratio

In [ ]:
df_user_ratio = (
    df_uk.query("UserType == 'New'")
    .groupby(["InvoiceYearMonth"])["CustomerID"]
    .nunique()
    / df_uk.query("UserType == 'Existing'")
    .groupby(["InvoiceYearMonth"])["CustomerID"]
    .nunique()
)
df_user_ratio = df_user_ratio.reset_index()
df_user_ratio = df_user_ratio.dropna()
df_user_ratio

In [ ]:
plot_data = [
    go.Bar(
        x=df_user_ratio.query("InvoiceYearMonth>201101 and InvoiceYearMonth<201112")[
            "InvoiceYearMonth"
        ],
        y=df_user_ratio.query("InvoiceYearMonth>201101 and InvoiceYearMonth<201112")[
            "CustomerID"
        ],
    )
]

plot_layout = go.Layout(xaxis={"type": "category"}, title="New Customer Ratio")
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

##### Monthly Retention Rate

Monthly Retention Rate = Retained Customers From Prev. Month/Active Customers Total

In [ ]:
df_user_purchase = (
    df_uk.groupby(["CustomerID", "InvoiceYearMonth"])["Revenue"].sum().reset_index()
)
df_user_purchase

In [ ]:
df_retention = pd.crosstab(
    df_user_purchase["CustomerID"], df_user_purchase["InvoiceYearMonth"]
).reset_index()
df_retention.head()

In [ ]:
months = df_retention.columns[2:]
months

In [ ]:
retention_array = []
for i in range(len(months) - 1):
    retention_data = {}
    selected_month = months[i + 1]
    prev_month = months[i]
    retention_data["InvoiceYearMonth"] = int(selected_month)
    retention_data["TotalUserCount"] = df_retention[selected_month].sum()
    retention_data["RetainedUserCount"] = df_retention[
        (df_retention[selected_month] > 0) & (df_retention[prev_month] > 0)
    ][selected_month].sum()
    retention_array.append(retention_data)

retention_array

In [ ]:
df_retention = pd.DataFrame(retention_array)
df_retention.head()

In [ ]:
df_retention["RetentionRate"] = (
    df_retention["RetainedUserCount"] / df_retention["TotalUserCount"]
)
df_retention

In [ ]:
plot_data = [
    go.Scatter(
        x=df_retention.query("InvoiceYearMonth<201112")["InvoiceYearMonth"],
        y=df_retention.query("InvoiceYearMonth<201112")["RetentionRate"],
        name="organic",
    )
]

plot_layout = go.Layout(xaxis={"type": "category"}, title="Monthly Retention Rate")
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

##### Cohort Based Retention Rate

In [ ]:
df_retention = pd.crosstab(
    df_user_purchase["CustomerID"], df_user_purchase["InvoiceYearMonth"]
).reset_index()

new_column_names = ["m_" + str(column) for column in df_retention.columns]
df_retention.columns = new_column_names

In [ ]:
retention_array = []
for i in range(len(months)):
    retention_data = {}
    selected_month = months[i]
    prev_months = months[:i]
    next_months = months[i + 1 :]
    for prev_month in prev_months:
        retention_data[prev_month] = np.nan

    total_user_count = retention_data["TotalUserCount"] = df_retention[
        "m_" + str(selected_month)
    ].sum()
    retention_data[selected_month] = 1

    query = "{} > 0".format("m_" + str(selected_month))

    for next_month in next_months:
        query = query + " and {} > 0".format(str("m_" + str(next_month)))
        retention_data[next_month] = np.round(
            df_retention.query(query)["m_" + str(next_month)].sum() / total_user_count,
            2,
        )
    retention_array.append(retention_data)

df_retention = pd.DataFrame(retention_array)
df_retention.index = months

In [ ]:
df_retention